# Pipeline

#### Tópicos da aula
- Transformers x Estimators
- Pipeline
- FunctionTransform
- ColumnTransform

___________________________


[Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) é uma classe do sklearn que permite aplicar uma sequência de transformações em um estimador final. <br>
Para isso, os passos intermediários devem ter implementados métodos de `fit` e `transform` enquanto o estimador final só precisa ter o `fit` implementado. <br>
O propósito do `pipeline` é:
- reunir várias etapas para serem validadas de forma cruzada (cross-validation) ao definir parâmetros diferentes
- ajudar a criar códigos que possuam um padrão que possa ser facilmente entendido e compartilhando entre times de cientista e engenheiro de dados.

<img src="images/pipeline.png" text="https://nbviewer.org/github/rasbt/python-machine-learning-book/blob/master/code/ch06/ch06.ipynb#Combining-transformers-and-estimators-in-a-pipeline">



- __Transformer:__ Um transformador se refere à um objeto de uma classe que possuim os métodos fit() e transform() e que nos ajudam a transformar o dado na forma que queremos. OneHotEncoder, SimpleImputer e MinMaxScaler são exemplos de transformers.
- __Estimator:__ Um estimador se refere à um algoritmo de ML. Ele é um objeto de uma classe que possui os métodos fit() e predict(). [Aqui](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) se encontram exemplos de estimadores.

Hoje vamos utilizar um dataset mais simples de exemplo. Usaremos os dados de gorjeta cuja descrição encontra-se [nesse link](https://vincentarelbundock.github.io/Rdatasets/doc/reshape2/tips.html).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Vamos importar o dataset
df = sns.load_dataset('tips')

# Add missing values -> pra dar uma graça no dataset
df.iloc[[1, 2, 4, 12], [2]] = np.nan
df.iloc[[10, 20, 40, 120, 222], [1]] = np.nan
df.iloc[[61, 27, 145, 212], [3]] = np.nan
df.iloc[[143, 237, 48, 102, 20], [4]] = np.nan
df.iloc[[71, 172, 194, 182], [5]] = np.nan
df.iloc[[83, 90, 33, 228], [6]] = np.nan
df

In [ ]:
# Vamos ver as dimensões dele
df.shape

In [ ]:
# Observe que cada feature tem pelo menos 1 nulo (afinal, non-null delas é menor que 150)
df.info()

In [ ]:
# Confirmando quantos nulos temos em cada coluna
df.isnull().sum()

Para as minhas features numéricas, eu quero seguir os seguintes passos de pré-processamento,
1. "padronizar" as minhas features (ou "normalizar", deixar elas com média 0 e desvio padrão 1),
2. adicionar a mediana em qualquer valor nulo,
3. treinar o algoritmo de ML.

Neste caso, note que eu preciso "treinar" os passos (1), (2) e (3) todos na base de treino, e depois só aplicar eles na base de validação.

In [ ]:
# particionando os dados
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['total_bill']), 
                                                    df['total_bill'], 
                                                    test_size=.2, 
                                                    random_state=42)

In [ ]:
# Para os passos de processamento das features, faremos todos juntos, com um Pipeline.
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

Como utilizar: <br>
O Pipeline é construído com uma lista de pares (key, value) nos quais a key é uma string que contém um nome para o step escolhido e o valor é o objeto da classe:

<img src="images/pipeline_sintaxe.png" width=500 />

tal que `transformacao_1` é o nome que você quer dar para essa etapa e `transformacao_1()` é a classe instanciada da transformação.
A lista com as transformações deve ser passada já com a sequencia em que elas devem ser aplicadas.
<br>
O Pipeline segue o mesmo framework do sklearn e por isso temos os métodos `.fit()`, `fit_transform()` e `.transform()` para os transformes e `.fit()` e `predict()` quando temos estimadores definidos dentro da sequência do pipeline.

In [ ]:
# Separa variáveis numéricas das categóricas
num_cols = X_train.select_dtypes("number").columns
cat_cols = X_train.select_dtypes(exclude="number").columns

# Cria nosso Pipeline com SimpleImputer, StandardScaler e KNeighborsRegressor


Os passos do pipeline podem ser acessados pelos índices ou passando a key:

In [ ]:
# por índice


In [ ]:
# por key


In [ ]:
# Podemos visualizar nosso pipe
from sklearn import set_config
set_config(display="diagram")
pipe_knn  # click on the diagram below to see the details of each step

In [ ]:
# Vamos rodar nosso pipeline no treino utilizando apenas as colunas numéricas


In [ ]:
# Fazendo o predict direto:


In [ ]:
# Agora avaliamos o modelo no nosso conjunto de validação.
print(f'MSE: {mean_squared_error(y_test, y_pred):.1f}')
print(f'MAE: {mean_absolute_error(y_test, y_pred):.1f}')

In [ ]:
# Tudo junto
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error


# particionando os dados
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(columns=['total_bill']), 
    df['total_bill'], 
    test_size=.2, 
    random_state=42
)

# Define variáveis numéricas e categóricas
num_cols = X_train.select_dtypes("number").columns
cat_cols = X_train.select_dtypes(exclude="number").columns

# Cria nosso Pipeline com SimpleImputer, StandardScaler e KNeighborsRegressor
pipe_knn = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('model', KNeighborsRegressor(n_neighbors=7))],
    verbose=True
)

# Vamos rodar nosso pipeline no treino
pipe_knn.fit(X_train[num_cols], y_train)

# Prediz no teste
y_pred = pipe_knn.predict(X_test[num_cols])

print()
print(y_pred[:10])
print()

# Agora avaliamos o modelo no nosso conjunto de validação.
print(f'MSE: {mean_squared_error(y_test, y_pred):.1f}')
print(f'MAE: {mean_absolute_error(y_test, y_pred):.1f}')

Por que isso é poderoso?

Imagina se quiséssemos fazer validação cruzada. Nesse caso, em cada fold que escolhermos como validação, teriamos que refazer cada passo na base de treino da vez,
1. achar a mediana daquela base de treino, e preencher os nulos,
2. achar a média e a variância daquela base de treino, para padronizar as features,
3. aí sim, treinamos o modelo na base de treino, e medimos a qualidade no fold de validação da vez.

Mas usando Pipeline, a gente não precisa fazer tudo passo a passo. O Pipeline se encarrega de fazer tudo de uma vez para nós. É como se o nosso "modelo" agora fosse o pipeline completo. 

In [ ]:
from sklearn.model_selection import cross_validate

### Tunando hiperparâmetros

In [ ]:
# Vamos checar quais os parâmetros que podemos utilizar


In [ ]:
# Tunando hiperparâmetros com 3-fold cross-validation e pipelines
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV



In [ ]:
# Agora podemos acessar os atributos do grid


E se eu tiver funções próprias ou queira aplicar alguma pronta do python?

## [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html)
Com a `FunctionTransformer` conseguimos adicionar funções que não possuem os métodos `.fit()` e `.transform()` ao pipeline. A função criada deve retornar um pandas Dataframe ou um array do numpy a fim de podermos utilizá-lo com o Pipelines.

In [ ]:
from sklearn.preprocessing import FunctionTransformer

# Para funções próprias
def somar_10(df):
    return df+10

def quebra_coluna(df):
    df['tip2'] = np.where(df['tip']<5, 0, 1)
    return df

def return_selected_cols(dataset, columns):
    return dataset[columns]



E como adicionar tratamento nas variáveis categóricas?

## [ColumnTransformer()](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html?highlight=columntransformer#sklearn.compose.ColumnTransformer)
Essa classe serve para __especificarmos em quais colunas a transformação deve ser aplicada__. Seu uso é bem simples, deve-se nomear o tratamento, especificar qual ele deve ser e especificar as colunas nas quais ele deve ser aplicado.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor

# Separa treino e teste
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['total_bill']), 
                                                    df['total_bill'], 
                                                    test_size=.2, 
                                                    random_state=42)

# Define colunas categóricas
cat_cols = list(X_train.select_dtypes('category').columns)
print(f"Colunas categoricas: {cat_cols}")

# Define colunas numéricas
num_cols = list(X_train.select_dtypes('number').columns)
print(f"Colunas numéricas: {num_cols}")

# Para funções próprias
def somar_10(df):
    return df+10

soma_10 = FunctionTransformer(somar_10, validate=False)

# Define pipeline numérico
num_pipe = Pipeline([
    ('imputer_cv', SimpleImputer(strategy='median')),
    ('log', FunctionTransformer(np.log)),
    ('soma10', soma_10),
    ('scaler_cv', StandardScaler()),
], verbose=True)

# Define pipeline categórico com SimpleImputer e OneHotEncoder
cat_pipe = Pipeline([
    ('imputer_cv', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')),
])

# Concatena pipelines categóricos e numéricos com suas respectivas colunas



# Define pipeline final com o preprocessor e o estimador



# Tunando hiperparâmetros com 3-fold cross-validation e pipelines
parameters = {'model__n_neighbors': [3, 4, 5],
              'model__p': [1,2],
              'model__weights': ["uniform", "distance"]}

kfold = KFold(n_splits=3, shuffle=True, random_state=42)
grid = GridSearchCV(pipeline, param_grid=parameters, cv=kfold, n_jobs=-1, return_train_score=True, scoring="neg_mean_squared_error")

grid.fit(X_train, y_train)

## Bibliografia e Aprofundamento
- [Python Machine Learning Book](https://github.com/rasbt/python-machine-learning-book-3rd-edition)
- [Documentação](https://scikit-learn.org/stable/modules/compose.html)
- [ColumnTransformer](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data)
- [FeatureUnion](https://scikit-learn.org/stable/modules/compose.html#featureunion-composite-feature-spaces)
- https://medium.com/data-hackers/como-usar-pipelines-no-scikit-learn-1398a4cc6ae9
- [Pipelines e funções próprias: FuncionTransformer](https://towardsdatascience.com/using-functiontransformer-and-pipeline-in-sklearn-to-predict-chardonnay-ratings-9b13fdd6c6fd)
- [Custom Functions: Como criar classes e usá-las no pipeline](https://tiaplagata.medium.com/how-scikit-learn-pipelines-make-your-life-so-much-easier-3cfbfa1d9da6)